In [0]:
from google.colab.drive import mount
mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
drive_path = 'drive/My Drive/'

In [0]:
!sudo apt-get install unar
!pip install pydicom

In [0]:
!cp 'drive/My Drive/INbreast.rar' INbreast.rar
!unar 'INbreast.rar'

In [0]:
!unar 'INbreast.rar'

In [0]:
import os
import pydicom 
import numpy as np 
import matplotlib.pyplot as plt
import cv2
from random import shuffle
import glob
from numpy.linalg import norm 
import pandas as pd
import csv

In [0]:
images = glob.glob('INbreast/AllDICOMs/*.dcm')

In [0]:
csv_path = 'INbreast/INbreast.xls'
df = pd.read_excel(csv_path)

filenames = df['File Name'].tolist()
labels   = df['Bi-Rads'].tolist()

data = dict(zip(filenames, labels))

In [0]:
import skimage.feature as ft

def get_id_from_mask(path):
  return path[:-9]

def get_image_from_id(id):
  img_paths = glob.glob('INbreast/AllDICOMs/*.dcm')
  for img_path in img_paths:
    if id in img_path:
      return img_path
    
def get_array_from_dicom(path):
  img = pydicom.dcmread(path).pixel_array
  return img

def get_mammo_from_mask(msk_path):
  id = get_id_from_mask(msk_path)
  img_path  = get_image_from_id(id)
  img = get_array_from_dicom(img_path)
  return img_path, img

def map_label(fname):
  for f in data.keys():
    l = int(f) 
    if str(l) in fname:
      label = data[f]
      break
  if label in ['4a', '4b', '4c']:
    label = 4
  
  return int(label) -2 

def preprocess(img, size = (256, 256)):
  img = cv2.resize(img, dsize = size)
  return img.astype('float32')

def resize_and_crop(img):
  result = np.zeros((1685, 1685))
  mask = img == 0
  rows = np.flatnonzero((~mask).sum(axis=1))
  cols = np.flatnonzero((~mask).sum(axis=0))

  x0 = rows.min()
  x1 = rows.max()+1
  y0 = cols.min()
  y1 = cols.max()+1
  
  v = img
  v = v[x0:x1, y0:y1]
  result[:v.shape[0],:v.shape[1]] = v
  return preprocess(result)

def hog(img, bin_n = 16, cellx = 16, celly = 16):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bin = np.int32(bin_n*ang/(2*np.pi))
    bin_cells = []
    mag_cells = []

    for i in range(0,img.shape[0]//celly):
        for j in range(0,img.shape[1]//cellx):
            bin_cells.append(bin[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])
            mag_cells.append(mag[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])   

    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)

    # transform to Hellinger kernel
    eps = 1e-7
    hist /= hist.sum() + eps
    hist = np.sqrt(hist)
    hist /= norm(hist) + eps

    return hist
  
def lbp(img, P = 16, R = 2, method = 'nri_uniform'):
  out = ft.local_binary_pattern(img, P, R, method)
  hist, _ = np.histogram(out, normed=True, bins=P+2, range=(0, P + 2))
  return hist

In [0]:
mass_msk_base = 'INbreast/extras/MassSegmentationMasks/'
mc_msk_base = 'INbreast/extras/CalcificationSegmentationMasks/'

msk_files = list(set(os.listdir(mass_msk_base))) #| set(os.listdir(mc_msk_base)))

features = []
labels = []

for msk_file  in msk_files:
  #get full mammogram 
  
  try:
    img_path, img = get_mammo_from_mask(msk_file)
  except:
    continue

  #check of MC view exists
  if msk_file in os.listdir(mc_msk_base):
    mc_msk = cv2.imread(mc_msk_base+msk_file)[...,0]
  else:
    mc_msk = np.zeros_like(img)

  #check if 
  if msk_file in os.listdir(mass_msk_base):
    mass_msk = cv2.imread(mass_msk_base+msk_file)[...,0]
  else:
    mass_msk = np.zeros_like(img) 


  #get the mc mask 
  label =  map_label(img_path)
  img = img/img.max()
  mass = mass_msk/255
  mc   = mc_msk/255

  processed_img = resize_and_crop(img*mass)

  hog_feature = hog(processed_img)
  #lbp_feature = lbp(processed_img)
  
  feature = np.concatenate([hog_feature], axis = 0)
  
  #print(hog_feature.shape, lbp_feature.shape)
  features.append(feature)
  labels.append(label)


In [0]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score          
from sklearn.model_selection import cross_validate

In [0]:
from sklearn import svm

scoring = {'acc': 'accuracy',
           'prec': 'precision_macro',
           'rec': 'recall_macro',
            'f1'  : 'f1_macro'}

clf = svm.LinearSVC(random_state=0)
scores = cross_validate(clf, features, labels, scoring=scoring, cv=10)
for score in list(scoring.keys()):
    key = f'test_{score}'
    print(score, np.mean(scores[key]))

In [14]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()
scores = cross_validate(clf, features, labels, scoring=scoring, cv=10)

for score in list(scoring.keys()):
    key = f'test_{score}'
    print(score, np.mean(scores[key]))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are 

acc 0.2114896214896215
prec 0.19042857142857142
rec 0.1923333333333333
f1 0.16699999999999998


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
scores = cross_validate(clf, features, labels, scoring=scoring, cv=10)
for score in list(scoring.keys()):
    key = f'test_{score}'
    print(score, np.mean(scores[key]))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dis

acc 0.34362637362637366
prec 0.21773809523809523
rec 0.2505833333333333
f1 0.2207961482961483


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=5, n_estimators=10)
scores = cross_validate(clf, features, labels, scoring=scoring, cv=10)

for score in list(scoring.keys()):
    key = f'test_{score}'
    print(score, np.mean(scores[key]))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dis

acc 0.4697313797313797
prec 0.268984126984127
rec 0.31858333333333333
f1 0.2844184704184704


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()
scores = cross_validate(clf, features, labels, scoring=scoring, cv=10)

for score in list(scoring.keys()):
    key = f'test_{score}'
    print(score, np.mean(scores[key]))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dis

acc 0.39655677655677657
prec 0.2234285714285714
rec 0.28274999999999995
f1 0.23583899433899433


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
features

In [0]:
out = np.concatenate([np.array(labels).reshape(-1, 1), np.array(features)], axis = 1)

In [0]:
np.savetxt('drive/My Drive/hog_features.csv', out, delimiter=',')